In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets
!pip install peft
!pip install evaluate scikit-learn transformers[torch]
!pip install accelerate>=0.26.0

Looking in indexes: https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


# Prepare the dataset

In [2]:
from datasets import load_dataset

imdb = load_dataset("stanfordnlp/imdb")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = imdb.map(tokenize_function, batched=True)


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
training_set = tokenized_datasets['train'].shuffle(42).select(range(5000))
shuffled_set =  tokenized_datasets['test'].shuffle(42)
validation_set  = shuffled_set.select(range(1000))
test_set = shuffled_set.select(range(1000, 2000))

## First method : Using the transformers library and pytorch


In [4]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

## Fine-tuning the whole model

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


## LoRA implementation

In [7]:
[k for k,v  in model.bert.named_parameters()]

['embeddings.word_embeddings.weight',
 'embeddings.position_embeddings.weight',
 'embeddings.token_type_embeddings.weight',
 'embeddings.LayerNorm.weight',
 'embeddings.LayerNorm.bias',
 'encoder.layer.0.attention.self.query.weight',
 'encoder.layer.0.attention.self.query.bias',
 'encoder.layer.0.attention.self.key.weight',
 'encoder.layer.0.attention.self.key.bias',
 'encoder.layer.0.attention.self.value.weight',
 'encoder.layer.0.attention.self.value.bias',
 'encoder.layer.0.attention.output.dense.weight',
 'encoder.layer.0.attention.output.dense.bias',
 'encoder.layer.0.attention.output.LayerNorm.weight',
 'encoder.layer.0.attention.output.LayerNorm.bias',
 'encoder.layer.0.intermediate.dense.weight',
 'encoder.layer.0.intermediate.dense.bias',
 'encoder.layer.0.output.dense.weight',
 'encoder.layer.0.output.dense.bias',
 'encoder.layer.0.output.LayerNorm.weight',
 'encoder.layer.0.output.LayerNorm.bias',
 'encoder.layer.1.attention.self.query.weight',
 'encoder.layer.1.attention.se

In [8]:
print(f"What is the model \n\n {model}")


What is the model 

 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

### Create the LoRA module

We will consider in this module the original linear, and the down and up projection

In [9]:
import torch
from torch import nn

class LoRALinear(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, rank: int):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.lora_down = nn.Linear(in_dim, rank, bias=False)
        self.lora_up = nn.Linear(rank, out_dim, bias=False)

        nn.init.zeros_(self.lora_up.weight)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x) + self.lora_up(self.lora_down(x))


### Create a function replacing linear by lora module
From the original Linear module return a LoRA module
* The linear of the LoRA linear must be initialised with pretrained weights

In [12]:
def linear_to_lora(linear, rank=8):
    has_bias = linear.bias is not None
    out_dim, in_dim = linear.weight.shape

    lora = LoRALinear(in_dim, out_dim, rank=rank).to(linear.weight.device, linear.weight.dtype)

    with torch.no_grad():
        lora.linear.weight.copy_(linear.weight)
        if has_bias:
            lora.linear.bias.copy_(linear.bias)

    return lora


We now replace the target linear by the LoRALinear described above

In [14]:
import copy

lora_model = copy.deepcopy(model)
lora_parameters = []

for layer in lora_model.bert.encoder.layer:
    layer.attention.self.query = linear_to_lora(layer.attention.self.query)
    layer.attention.self.key = linear_to_lora(layer.attention.self.key)
    layer.attention.self.value = linear_to_lora(layer.attention.self.value)
    layer.attention.output.dense = linear_to_lora(layer.attention.output.dense)
    layer.intermediate.dense = linear_to_lora(layer.intermediate.dense)
    layer.output.dense = linear_to_lora(layer.output.dense)

    lora_parameters += list(layer.attention.self.query.lora_down.parameters())
    lora_parameters += list(layer.attention.self.query.lora_up.parameters())
    lora_parameters += list(layer.attention.self.key.lora_down.parameters())
    lora_parameters += list(layer.attention.self.key.lora_up.parameters())
    lora_parameters += list(layer.attention.self.value.lora_down.parameters())
    lora_parameters += list(layer.attention.self.value.lora_up.parameters())
    lora_parameters += list(layer.attention.output.dense.lora_down.parameters())
    lora_parameters += list(layer.attention.output.dense.lora_up.parameters())
    lora_parameters += list(layer.intermediate.dense.lora_down.parameters())
    lora_parameters += list(layer.intermediate.dense.lora_up.parameters())
    lora_parameters += list(layer.output.dense.lora_down.parameters())
    lora_parameters += list(layer.output.dense.lora_up.parameters())


In [15]:
print(f"What is the model modified with LoRA: \n\n {lora_model}")


What is the model modified with LoRA: 

 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): LoRALinear(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora_down): Linear(in_features=768, out_features=8, bias=False)
                (lora_up): Linear(in_features=8, out_features=768, bias=False)
              )
              (key): LoRALinear(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora

### Defining module requiring grad

In [16]:
for k,v  in lora_model.bert.named_parameters():
    print(k)
    if ('lora' in k):
        v.requires_grad = True
    else:
        v.requires_grad = False

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.linear.weight
encoder.layer.0.attention.self.query.linear.bias
encoder.layer.0.attention.self.query.lora_down.weight
encoder.layer.0.attention.self.query.lora_up.weight
encoder.layer.0.attention.self.key.linear.weight
encoder.layer.0.attention.self.key.linear.bias
encoder.layer.0.attention.self.key.lora_down.weight
encoder.layer.0.attention.self.key.lora_up.weight
encoder.layer.0.attention.self.value.linear.weight
encoder.layer.0.attention.self.value.linear.bias
encoder.layer.0.attention.self.value.lora_down.weight
encoder.layer.0.attention.self.value.lora_up.weight
encoder.layer.0.attention.output.dense.linear.weight
encoder.layer.0.attention.output.dense.linear.bias
encoder.layer.0.attention.output.dense.lora_down.weight
encoder.layer.0.attention.output.dense.lora_up.weight
encoder.la

## Using transformers Trainer to fine-tune with LoRA 

In [17]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_custom_lora",
                                  eval_strategy="steps",
                                  eval_steps= 128,
                                  num_train_epochs=2,)
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset= training_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)


In [19]:
trainer.train()

/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
128,No log,1.260714,0.681000
256,No log,0.755160,0.849000
384,No log,0.729394,0.853000
512,0.953346,0.633637,0.867000


/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=626, training_loss=0.8810046138093114, metrics={'train_runtime': 374.1155, 'train_samples_per_second': 26.73, 'train_steps_per_second': 1.673, 'total_flos': 2671879188480000.0, 'train_loss': 0.8810046138093114, 'epoch': 2.0})

## Using PEFT library to fine-tune with LoRA 

In [20]:
from peft import LoraConfig, TaskType,  get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)
peft_model = get_peft_model(model, lora_config)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_custom_lora",
                                  eval_strategy="steps",
                                  eval_steps= 128,
                                  num_train_epochs=2,)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset= training_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
128,No log,1.356935,0.636000
256,No log,1.333106,0.637000
384,No log,1.314133,0.640000
512,1.368475,1.301800,0.649000


/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/people/dicko/miniconda3/envs/DataSR/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.wa

TrainOutput(global_step=626, training_loss=1.361582064400085, metrics={'train_runtime': 329.1375, 'train_samples_per_second': 30.382, 'train_steps_per_second': 1.902, 'total_flos': 2632290263040000.0, 'train_loss': 1.361582064400085, 'epoch': 2.0})

## Compare the results of the different approaches